In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

In [47]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')
df_48ind.replace(-99.99, np.nan, inplace = True)
df_48ind.replace(-999, np.nan, inplace = True)
df_48ind.dropna(inplace = True)

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


In [49]:
df_48ind

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1969-07-01,-8.32,-6.92,-3.53,-2.10,-4.74,-7.87,-10.15,-11.07,-3.43,-11.91,...,-3.15,-12.50,-10.09,-8.19,-9.93,-6.60,-10.18,-12.74,-9.05,-14.64
1969-08-01,4.23,4.52,5.44,6.63,3.99,6.42,11.56,9.79,4.77,2.40,...,6.08,5.59,6.38,8.21,8.98,12.76,8.80,15.06,8.89,8.38
1969-09-01,1.59,-0.58,5.28,-0.91,4.12,2.13,0.57,-0.61,-0.52,-2.64,...,1.23,-1.56,0.21,-0.75,-1.35,-2.94,7.08,-3.79,-2.54,-6.67
1969-10-01,16.95,7.97,6.84,11.64,12.42,6.28,13.57,7.59,6.41,6.72,...,3.50,4.89,11.25,6.25,9.03,10.71,5.71,4.54,10.31,3.70
1969-11-01,-1.87,-0.66,-4.91,-2.83,0.61,-8.33,-3.27,-3.97,-2.42,-2.90,...,-1.98,-7.91,-6.83,-4.08,-1.90,-8.99,-4.90,-6.23,-5.91,-2.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.84,-4.28,-6.43,-5.13,-2.43,-7.38,-11.17,-6.19,-6.81,-4.41,...,-2.57,-5.15,-2.69,-6.16,-4.45,-3.92,2.57,-10.45,-3.49,-2.72
2023-10-01,-5.83,-4.98,-0.39,-4.34,-4.00,-14.31,2.77,-2.05,-0.05,4.21,...,-3.46,-6.31,-0.90,0.74,-1.71,-2.89,3.49,-9.05,-6.31,-1.77
2023-11-01,-2.29,4.26,4.79,3.90,4.76,5.66,11.08,7.90,4.97,10.63,...,10.43,12.31,7.94,6.96,7.67,13.16,3.93,9.95,13.51,6.64


## 1) La capitalisation boursière en tant que taille moyenne de l'entreprise x nombre d'entreprises

In [50]:
market_cap = df_numfirm.multiply(df_avgsize, axis=0)
market_cap = market_cap.loc[df_48ind.index]
market_cap

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1969-07-01,293.60,16916.90,6142.08,3003.30,3735.82,4728.28,2214.96,4315.95,51336.92,4571.00,...,11522.60,14806.97,3010.15,33087.75,4481.12,10436.16,6617.92,1579.41,10901.25,136.80
1969-08-01,267.45,15709.85,5924.52,2937.45,3553.88,4355.90,1989.12,3838.01,49540.16,4018.00,...,11148.26,12940.99,2695.00,30332.40,4033.68,9728.46,5929.28,1377.70,9896.40,116.24
1969-09-01,278.45,16370.90,6242.04,3123.15,3656.29,4628.09,2219.04,4200.95,51800.60,4076.52,...,11786.10,13579.31,2861.10,32669.52,4394.88,10956.54,6225.75,1583.78,10742.40,126.00
1969-10-01,282.85,16231.80,6541.78,3059.55,3800.94,4717.53,2229.60,4170.13,51452.84,3958.53,...,11928.90,13345.47,2861.65,32401.20,4329.08,10573.92,6657.30,1523.06,10449.90,117.20
1969-11-01,329.05,17492.15,6988.24,3411.60,4268.11,5013.37,2530.08,4486.15,54661.88,4215.90,...,12323.98,13985.37,3182.30,34368.36,4718.56,11674.34,7018.20,1591.83,11505.15,121.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,41006.68,505297.52,346342.32,346278.68,229020.70,41627.04,394473.12,27397.08,677271.34,252346.56,...,75465.50,812140.55,519543.20,3359902.26,696424.79,2626741.91,1581703.20,68858.24,1204022.82,1219430.46
2023-10-01,41351.20,482695.72,322070.22,328411.27,219676.30,38554.32,350182.43,25643.28,630874.36,240484.16,...,73352.80,769734.55,505316.24,3152332.77,665101.66,2520866.93,1618384.68,61585.28,1159465.58,1184147.60
2023-11-01,38939.89,453495.00,320819.40,314148.67,210891.15,32941.16,359816.49,25072.20,626535.45,250580.48,...,70811.40,721091.80,497719.17,3174484.60,653520.80,2442166.45,1674202.86,55927.10,1082860.33,1162998.24


## 2) Le rapport entre la valeur comptable (BE : book equity) et la valeur de marché (ME : market equity), c'est-à-dire le rapport book-to-market, en utilisant les données "Sum of BE / Sum of ME".

### (Remarque : les données "Somme de BE / Somme de ME" sont annuelles et doivent être converties en données mensuelles en supposant que le ratio reste constant entre juillet de l'année s et juin de l'année s + 1. Par exemple, le ratio de 1926 est le ratio qui doit être utilisé pour la période allant de juillet 1926 à juin 1927).

In [53]:
df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)

df_BtoM

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1969-07-01,0.88,0.50,0.2,0.58,0.66,0.31,0.42,0.33,0.21,0.50,...,0.36,1.32,0.49,0.41,0.24,0.60,0.54,0.27,0.74,0.60
1969-08-01,0.88,0.50,0.2,0.58,0.66,0.31,0.42,0.33,0.21,0.50,...,0.36,1.32,0.49,0.41,0.24,0.60,0.54,0.27,0.74,0.60
1969-09-01,0.88,0.50,0.2,0.58,0.66,0.31,0.42,0.33,0.21,0.50,...,0.36,1.32,0.49,0.41,0.24,0.60,0.54,0.27,0.74,0.60
1969-10-01,0.88,0.50,0.2,0.58,0.66,0.31,0.42,0.33,0.21,0.50,...,0.36,1.32,0.49,0.41,0.24,0.60,0.54,0.27,0.74,0.60
1969-11-01,0.88,0.50,0.2,0.58,0.66,0.31,0.42,0.33,0.21,0.50,...,0.36,1.32,0.49,0.41,0.24,0.60,0.54,0.27,0.74,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.65,0.43,0.1,0.13,0.28,0.39,0.17,0.59,0.14,0.19,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
2023-10-01,0.65,0.43,0.1,0.13,0.28,0.39,0.17,0.59,0.14,0.19,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
2023-11-01,0.65,0.43,0.1,0.13,0.28,0.39,0.17,0.59,0.14,0.19,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48


## 3) Le momentum de chaque industrie est le rendement moyen de cette industrie au cours des 12 derniers mois, y compris le mois t.

In [46]:
# Calculer la moyenne mobile sur 12 mois pour chaque industrie
df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)

df_mom

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,-1.608333,-1.192500,-0.079167,-0.121667,1.379167,-4.351667,-2.588333,-4.757500,-1.919167,-4.050000,...,-2.296667,-4.505000,-4.432500,-2.108333,-4.271667,-0.481667,-1.880000,-4.616667,-3.060833,-7.690833
1970-07-01,-0.982500,-0.151667,0.701667,0.616667,2.065833,-2.652500,-1.698333,-2.729167,-1.250000,-2.367500,...,-1.479167,-2.764167,-3.231667,-0.444167,-3.360833,0.620833,-0.150000,-2.749167,-1.442500,-6.460833
1970-08-01,-1.360833,-0.497500,0.105833,-0.161667,1.787500,-2.236667,-2.392500,-3.003333,-1.295833,-1.963333,...,-1.340000,-2.526667,-3.212500,-0.702500,-3.523333,0.090000,-0.750833,-3.533333,-1.712500,-5.955000
1970-09-01,-0.289167,-0.032500,0.001667,0.286667,1.486667,-1.895833,-0.841667,-2.345000,-0.720833,-0.746667,...,-1.426667,-1.650000,-1.866667,-0.072500,-2.071667,0.301667,-0.614167,-1.979167,-0.775833,-4.920000
1970-10-01,-1.998333,-0.698333,-0.209167,-0.768333,1.314167,-2.781667,-2.284167,-3.220000,-1.268333,-1.613333,...,-1.936667,-2.470000,-3.435833,-0.665833,-3.153333,-1.025833,-1.920833,-3.140000,-2.250833,-6.836667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-0.443333,0.130833,0.800833,0.579167,1.386667,0.681667,2.873333,2.539167,1.005833,1.811667,...,1.295000,1.810000,2.147500,1.186667,1.638333,0.859167,0.887500,1.377500,1.430000,2.147500
2023-10-01,-2.074167,-1.222500,0.175000,-0.571667,0.047500,-0.840000,1.854167,1.636667,0.530000,1.255833,...,0.534167,0.715833,0.934167,1.092500,0.615000,-0.496667,0.110000,0.045833,-0.054167,1.101667
2023-11-01,-2.465833,-1.114167,-0.067500,-0.516667,-0.021667,-0.790833,2.170833,0.989167,0.159167,0.682500,...,0.526667,0.895000,1.174167,1.427500,0.755000,0.169167,0.320833,-0.030833,0.355000,1.069167


## Pour chaque mois t, standardiser chaque caractéristique de manière transversale pour avoir une moyenne nulle et un écart-type unitaire pour toutes les actions à la date t, comme expliqué dans Brandt et al. (2009).

In [6]:
# Créer un objet StandardScaler
scaler = StandardScaler()

def standardize(df):
    # Standardiser les données
    df_scaled = scaler.fit_transform(df)
    
    df_standardized = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)
    
    return df_standardized

standardized_market_caps = standardize(market_cap)
standardized_BtoM = standardize(df_BtoM)
standardized_mom = standardize(df_mom)

market_weights = market_cap.div(market_cap.sum(axis=1), axis=0)

In [20]:
optimal_weights = market_weights + (1 / len(market_weights.columns)) * (optimal_beta[0] * standardized_market_caps + optimal_beta[1] * standardized_BtoM + optimal_beta[2] * standardized_mom)

optimal_weights.dropna(inplace = True)
optimal_weights

# Code original

In [30]:
import numpy as np
from scipy.optimize import minimize

def u(x, gamma):
    return (x ** (1 - gamma) - 1) / (1 - gamma)

# Fonction à maximiser
def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma):
    wi_t = np.random.rand(N) # Supposons que vous avez une fonction pour calculer wi_t
    return -(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)), gamma) * (1 + r))

# Données
T = 54  # Nombre d'observations
N = 48   # Nombre d'actifs
MC = np.random.rand(T, N)  # Exemple de données de market cap
BM = np.random.rand(T, N)  # Exemple de données de book-to-market ratio
MOM = np.random.rand(T, N) # Exemple de données de momentum
r = np.random.rand(T)       # Exemple de données de rendement
gamma = 0.5  # Paramètre de la fonction d'utilité CRRA

# Initialisation des coefficients beta
initial_beta = np.array([0.5, 0.5, 0.5])

# Optimisation des coefficients beta
result = minimize(function_to_maximize, initial_beta, args=(MC, BM, MOM, r, T, N, gamma), method='SLSQP')

# Résultats
optimal_beta = result.x
maximized_value = -result.fun

print("Coefficients β optimaux :", optimal_beta)
print("Valeur maximisée de la fonction :", maximized_value)


Coefficients β optimaux : [671194.56469017 380780.8218448  690714.58475625]
Valeur maximisée de la fonction : 20791.298805983857


/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_3582/1715546139.py:5: RuntimeWarning: invalid value encountered in scalar power
  return (x ** (1 - gamma) - 1) / (1 - gamma)


# ******

In [33]:
def maximize_utility(MC, BM, MOM, r, T, N, gamma, start, end):
    
    def u(x, gamma):
        return x ** (1 - gamma) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma):
        wi_t = market_weights.loc[start:end]
        return -(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC.loc[start:end] + beta[1] * BM.loc[start:end] + beta[2] * MOM.loc[start:end])), gamma) * (1 + r))

    # Initialisation des coefficients beta
    initial_beta = np.array([0.5, 0.5, 0.5])

    # Optimisation des coefficients beta
    result = minimize(function_to_maximize, initial_beta, args=(MC, BM, MOM, r, T, N, gamma), method='SLSQP')

    # Résultats
    optimal_beta = result.x

    return optimal_beta

start_date = pd.to_datetime('1969-07-01')
end_date = pd.to_datetime('1973-12-01')

maximize_utility(standardized_market_caps, standardized_BtoM, standardized_mom, np.random.rand(48), 48, 48, 0.5, start_date, end_date)

array([-7.18817049e+10,  5.17125394e+10, -6.89966157e+10])

In [36]:
import numpy as np
from scipy.optimize import minimize

def maximize_utility(MC, BM, MOM, r, T, N, gamma):

    def u(x, gamma):
        return x ** (1 - gamma) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma, wi_t):
        return -(1/T) * u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)), gamma) * (1 + r)

    # Initialisation des coefficients beta
    initial_beta = np.array([0.5, 0.5, 0.5])

    optimal_betas = []

    for i in range(len(MC)):
        wi_t = market_weights[i]  # Accédez directement à l'élément i
        result = minimize(function_to_maximize, initial_beta, args=(MC[i], BM[i], MOM[i], r, T, N, gamma, wi_t), method='SLSQP')
        optimal_betas.append(result.x)


    return optimal_betas

start_date = pd.to_datetime('1969-07-01')
end_date = pd.to_datetime('1973-12-01')


mw = market_weights.loc[start_date:end_date]
MC = standardized_market_caps.loc[start_date:end_date]
MC = standardized_market_caps.loc[start_date:end_date]
MC = standardized_market_caps.loc[start_date:end_date]
r=df_48ind.loc[start_date:end_date].shift(-1)


maximize_utility(MC, BM, MOM, r, T, N, gamma)

KeyError: 0

In [ ]:
optimal_weights = market_weights + (1 / len(market_weights.columns)) * (optimal_beta[0] * standardized_market_caps + optimal_beta[1] * standardized_BtoM + optimal_beta[2] * standardized_mom)

optimal_weights.dropna(inplace = True)
optimal_weights